In [1]:
%%capture
!pip install gpjax
!pip install beartype
!pip install -U tensorflow_probability

In [2]:
import jax

jax.config.update("jax_enable_x64", True)
from beartype.typing import Callable, Tuple
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import optax as ox
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gpjax as gpx
from jaxtyping import Float64, Array
import beartype.typing as tp
from beartype import beartype

plt.style.use(
    "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
)
cols = plt.rcParams["axes.prop_cycle"].by_key()["color"]
key = jr.PRNGKey(123)

## Unit Tests and Typing

In the previous demo, we wrote a function to compute the Gram matrix of a kernel.

## Exercise 1

We have the following types: `Float64`, `Array`, `tp.Dict`, `str`, and `Callable` available in the notebook. Using these types, annotate the arguments and return types of the `rbf_kernel` and `evaluate_kernel` functions below.

Notes

1. `Callable` is synonymous to a function and has the syntax `Callable[[input_type_1, input_type_2], return_type]`.
2. An $N\times D$ Float64 array is annotated `Float64[Array, "N D"]`.


In [3]:
@beartype
def rbf_kernel(x, y, params):
    ell, sigma = params["lengthscale"], params["variance"]
    tau = jnp.sum(jnp.square(x / ell - y / ell))
    return sigma * jnp.exp(-0.5 * tau)


@beartype
def evaluate_kernel(kernel_fn, x, y, params):
    K = jax.vmap(lambda x1: jax.vmap(lambda y1: kernel_fn(x1, y1, params))(y))(x)
    return K.squeeze()


N = 100
X = jnp.sort(jr.uniform(key, shape=(N, 1), minval=-3.0, maxval=3), axis=0)
params = {"lengthscale": jnp.array([1.0]), "variance": jnp.array([0.5])}
K = evaluate_kernel(rbf_kernel, X, X, params)

(100, 100)

Let's now write a function to test our kernel function. Due to Beartype and JaxTyping, we do not need to worry about types or shapes. However, we should check that the kernel is being _correctly_ computed.

## Exercise 2

Write a unit test to determine if the kernel matrix produced by `evaluate_kernel` is positive definite.


In [4]:
%%file compute_kernel.py
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jaxtyping import Float64, Array


def rbf_kernel(x, y, params):
    ell, sigma = params["lengthscale"], params["variance"]
    tau = jnp.sum(jnp.square(x / ell - y / ell))
    return sigma * jnp.exp(-0.5 * tau)


def evaluate_kernel(kernel_fn, x, y, params):
    K = jax.vmap(lambda x1: jax.vmap(lambda y1: kernel_fn(x1, y1, params))(y))(x)
    return K.squeeze()


def stabilise_covariance(K, jitter):
    N = K.shape[0]
    eye = jnp.eye(N)
    jitter_matrix = eye * jitter
    return K + jitter_matrix

Overwriting compute_kernel.py


In [5]:
%%file test_kernel.py

from compute_kernel import evaluate_kernel, rbf_kernel, stabilise_covariance
import pytest
import jax.random as jr
import jax.numpy as jnp


@pytest.mark.parametrize('n', [10, 100])
@pytest.mark.parametrize('lengthscale', [0.1, 1.])
@pytest.mark.parametrize('variance', [0.1, 1.])
def test_kernel_psd(n, lengthscale, variance):
    pass

Overwriting test_kernel.py


In [6]:
!python -m pytest test_kernel.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: jaxtyping-0.2.25, typeguard-2.13.3, anyio-3.7.1
collected 8 items                                                                                  

test_kernel.py ........                                                                      [100%]

======================================== 8 passed in 2.42s =========================================


## Using GPJax

In the previous demo we saw how one can implement a GP using JAX. In this demo we're going to fit more complex regression models using GPJax. To get us started, let's first simulate a training dataset. To do this, we'll first define a GP prior using GPJax, and then draw a single sample from it.


In [ ]:
N = 100
true_lengthscale = 0.75
true_variance = 0.5
true_noise = 0.1
kernel = gpx.kernels.Matern32(lengthscale=true_lengthscale, variance=true_variance)

# The points at which we'd like to evaluate the prior
X = jnp.sort(jr.uniform(key, shape=(N, 1), minval=-3.0, maxval=3.0), axis=0)

# Defining the prior
_prior = gpx.gps.Prior(mean_function=gpx.mean_functions.Zero(), kernel=kernel)
prior_mvn = _prior.predict(X)  # A 100-dimensional Gaussian
prior_sample = prior_mvn.sample(seed=key, sample_shape=(1,)).T

# Applying iid Gaussian noise to the prior sample
key, subkey = jr.split(key)
noise = jr.normal(subkey, shape=(N, 1)) * true_noise
y = prior_sample + noise

# Plot our sample
fig, ax = plt.subplots()
ax.plot(X, y, "x", label="Observations")
ax.plot(X, prior_sample, label="Latent sample")
ax.legend()

## Fitting a GP Regression Model

To instantiate the GP prior in GPJax, we must specify the mean function and kernel that parameterise our GP prior. With this, we are then able to define the prior.

For the posterior, we must also specify the likelihood function. As we are carrying out GP regression in this example, a Gaussian likelihood will be assumed. Once we have a prior and a likelihood, we are able to instantiate the posterior distribution. We see this below.


In [8]:
kernel = gpx.kernels.RBF()
meanf = gpx.mean_functions.Zero()
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)

likelihood = gpx.likelihoods.Gaussian(num_datapoints=N)

posterior = prior * likelihood

Now that we have a posterior distribution defined, we would like to optimise its parameters and obtain Type-II MLEs, as we did in the first notebook. Achieving this in GPJax is straightforward as we simply instantiate the `ConjugateMLL` object and evaluate it for a given posterior distribution and dataset.


In [9]:
D = gpx.Dataset(X=X, y=y)
negative_mll = gpx.objectives.ConjugateMLL(negative=True)

negative_mll(posterior, train_data=D)

Array(104.37393914, dtype=float64)

We'll use GPJax's convenient `fit` function to optimise the parameters of our GP. Under-the-hood, this is performing gradient descent in a very similar manner to what we saw in the first demo.


In [10]:
import optax as ox

opt = ox.adam(learning_rate=0.01)

opt_posterior, history = gpx.fit(
    model=posterior,
    objective=jax.jit(negative_mll),
    train_data=D,
    optim=opt,
    key=key,
    num_iters=500,
)

  0%|          | 0/500 [00:00<?, ?it/s]

We can inspect the MLL that was obtained at each step of the optimisation routine to check for convergence.


In [ ]:
fig, ax = plt.subplots()
ax.plot(history, label="MLL")
ax.set(xlabel="Iteration", ylabel="MLL")
ax.legend()

Looks good! Now let's compute the predictive posterior distribution for our GP. We'll do this by first defining a test dataset, and then using the `predict` function. This function takes in a posterior distribution and a test dataset, and returns the predictive posterior distribution for the test dataset.


In [12]:
xtest = jnp.linspace(-3.1, 3.1, 500)[:, None]

latent_dist = opt_posterior.predict(xtest, train_data=D)
predictive_dist = opt_posterior.likelihood(latent_dist)

predictive_mean = predictive_dist.mean()
predictive_std = predictive_dist.stddev()

We can also plot the distribution, as we did before.


In [ ]:
def plot_predictions(X, y, xtest, predictive_mean, predictive_std):
    fig, ax = plt.subplots(figsize=(7.5, 2.5))
    ax.plot(X, y, "x", label="Observations", color=cols[0], alpha=0.5)
    ax.fill_between(
        xtest.squeeze(),
        predictive_mean - 2 * predictive_std,
        predictive_mean + 2 * predictive_std,
        alpha=0.2,
        label="Two sigma",
        color=cols[1],
    )
    ax.plot(
        xtest,
        predictive_mean - 2 * predictive_std,
        linestyle="--",
        linewidth=1,
        color=cols[1],
    )
    ax.plot(
        xtest,
        predictive_mean + 2 * predictive_std,
        linestyle="--",
        linewidth=1,
        color=cols[1],
    )
    ax.plot(xtest, predictive_mean, label="Predictive mean", color=cols[1])
    ax.legend(loc="center left", bbox_to_anchor=(0.975, 0.5))


plot_predictions(X, y, xtest, predictive_mean, predictive_std)

## Exercise 3

Propose alternative parameterisations of your Gaussian process and fit them. Compare the final marginal log-likelihood values that you obtain. What do you notice? When is the marginal log-likelihood maximised. Note: the printout from `fit` is the negative MLL, so here you'd seek the minimiser.


## Modelling Real-World Data

We support a large number of kernels in GPJax. A convenient property of kernels is that they are closed under addition and multiplication. This allows us to combine kernels to capture more complex structures in the data. In this section, we shall do this to model the Mauna Loa dataset. To get started, let's first load and inspect the data.


In [ ]:
co2_data = pd.read_csv(
    "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv", comment="#"
)
co2_data = co2_data.loc[co2_data["decimal date"] < 1990 + 11 / 12]

# Partition into a train and test set
Ntrain = int(co2_data.shape[0] * 0.8)
Xtr = co2_data["decimal date"].values[:Ntrain, None]
Xte = co2_data["decimal date"].values[Ntrain:, None]
ytr = co2_data["average"].values[:Ntrain, None]
yte = co2_data["average"].values[Ntrain:, None]

# Standardise the response
yscaler = StandardScaler()
ytr = yscaler.fit_transform(ytr)
yte = yscaler.transform(yte)

# Store the data in a Dataset object
co2_data = gpx.Dataset(X=Xtr, y=ytr)

# Plot the data
fig, ax = plt.subplots()
ax.plot(Xtr, ytr, label="Train")
ax.plot(Xte, yte, label="Test")
ax.set_title("CO2 Concentration in the Atmosphere")
ax.set_xlabel("Year")
ax.set_ylabel("CO2 Concentration (ppm)")
ax.legend()

Clearly there's some complex correlation present in this data, but let's try a simple stationary kernel first to see how it performs.


In [ ]:
# Define the prior
kernel = gpx.kernels.Matern52()
meanf = gpx.mean_functions.Constant()
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)

# Instantiate the likelihood and posterior
likelihood = gpx.likelihoods.Gaussian(num_datapoints=co2_data.n)
posterior = prior * likelihood

# Optimise the posterior w.r.t. the MLL
mll = jax.jit(gpx.objectives.ConjugateMLL(negative=True).step)
opt_posterior, history = gpx.fit(
    model=posterior,
    objective=mll,
    train_data=co2_data,
    optim=ox.adamw(learning_rate=0.1),
    num_iters=500,
    key=key,
)

# Let's predict at all points
latent_dist = opt_posterior.predict(jnp.vstack([Xtr, Xte]), train_data=co2_data)
predictive_dist = opt_posterior.likelihood(latent_dist)

predictive_mean = predictive_dist.mean()
predictive_std = predictive_dist.stddev()

plot_predictions(
    jnp.vstack([Xtr, Xte]),
    jnp.vstack([ytr, yte]),
    jnp.vstack([Xtr, Xte]),
    predictive_mean,
    predictive_std,
)

Clearly this is not the behaviour that we would like from our GP.

## Exercise 4

Using the above, try and design a better kernel and mean function for the data. Note that two kernels can be summed or multiplied in GPJax using the `+` and `*` operators respectively. To see the list of available kernels you may print the `gpx.kernels.__all__` variable.
